## Classiq's YQuantum 2024 Sparse State-Preparation Challenge

Welcome to the Sparse State-Preparation Challenge, presented by Classiq at YQuantum 2024. This challenge is designed for quantum computing enthusiasts of all levels, aiming to enhance your understanding and spur innovation in sparse state-preparation.

### Challenge Overview
- **Objective**: Develop quantum circuits that efficiently prepare sparse quantum states. These states are characterized by a small number of non-zero amplitudes, reducing quantum resource requirements and enhancing practical execution on near-term quantum hardware.

### Background
- **Importance of State Preparation**: State preparation is a crucial first step for many quantum algorithms. It involves setting a quantum state to represent a specific vector of probabilities or functions. The challenge focuses on sparse states, which are pivotal in applications like quantum linear algebra and quantum machine learning.

### Problem Statement
- **Sparse State-Preparation**: Your task is to create circuits that load a quantum state with probabilities corresponding to a given sparse vector. This challenge highlights the nuances of designing circuits that are both efficient and effective for today's quantum computing limitations.

### Resources and Guidance
- **Classiq's Tools**: Utilize Classiq’s platform, QMOD language, and SDK, which provide robust tools for designing and testing quantum algorithms. Participants are encouraged to base their implementations on "An Efficient Algorithm for Sparse Quantum State Preparation" by Niels Gleinig and Torsten Hoefler.

### Goals and Expectations
- **Output Goal**: Efficiently prepare a quantum state reflecting specified probabilities, ensuring high fidelity and resource efficiency. 
- **Example**: For input `{‘00000001’: 0.25, ‘00010001’: 0.5, ‘11001000’: 0.25}`, the output quantum state should correspond to these probabilities.


In [3]:
import classiq
classiq.authenticate()

c:\Users\vidur\AppData\Local\Programs\Python\Python310\lib\site-packages\classiq\_internals\authentication\token_manager.py:96: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
from classiq import *
import numpy as np

## Classical Functions
This section includes utility functions for classical pre-processing in quantum algorithms:

- `dict_to_3d_array(sparse_states)`: Converts a dictionary of sparse states into a 3D array format.
- `custom_filter(func, iterable)`: Filters items in an iterable based on a function.
- `unequal_sets(t, n)`: Determines the best qubit to split a set `t` into subsets with a significant size difference.
- `process_subsets(t, n, dif_qubits, dif_values)`: Processes subsets to determine difference qubits and values.
- `toggle_operations(index, n, x_x, ops1, ops2, s)`: Toggles operations based on the index condition.
- `conditional_toggle(ops1, ops2, n, dif, b, s)`: Conditionally toggles based on a difference.
- `calc_alpha_beta(x_1, x_2)`: Calculates and adjusts alpha and beta values.

In [10]:
def dict_to_3d_array(sparse_states):
    array_3d = []
    
    for key, value in sparse_states.items():
        point = [int(digit) for digit in key]
        array_3d.append([point, value])
    
    return array_3d

def custom_filter(func, iterable):
    return [item for item in iterable if func(item)]

def unequal_sets(t, n): 

    best_qubit = None
    T_0=[]
    T_1=[]
    current_difference = float('-inf')

    for b in range(n):    
        #Filter list based on boolean condition
        T_0 = custom_filter(lambda x, b=b: x[0][b] == 0, t)
        T_1 = custom_filter(lambda x, b=b: x[0][b] == 1, t)

        # Check if both sets are non-empty
        if len(T_0) != 0 and len(T_1) != 0:
            difference = abs(len(T_0) - len(T_1))
            #If new max difference
            if difference > current_difference:
                current_difference = difference
                best_qubit = b
                t_0 = T_0
                t_1 = T_1
    
    return best_qubit,t_0, t_1

def process_subsets(t, n, dif_qubits, dif_values):
    while len(t) > 1:
        b, T_0, T_1 = unequal_sets(t, n)
        dif_qubits.append(b)
        if len(T_0) < len(T_1):
            t = T_0
            dif_values.append(0)
        else:
            t = T_1
            dif_values.append(1)
    return dif_qubits, dif_values, t

def toggle_operations(index,n, x_x, ops1, ops2,s):
    if x_x[0][index] != 1: #Identical code
            ops1 += [1]
            ops2 += [n-1-index]
            for x in s:
                x[0][index]= int(not x[0][index])

def conditional_toggle(ops1,ops2,n,dif,b,s):
    ops1 += [2]
    sx = [n-1-dif,n-1-b]
    ops2 += [sx]
    for x in s:
        if x[0][dif] == 1:
            x[0][b] = int(not x[0][b])

def calc_alpha_beta(x_1,x_2):
    beta = x_1[1]
    alpha = x_2[1]

    x_2[1] = alpha+beta
    alpha = alpha/x_2[1]
    beta = beta/x_2[1]

    return alpha, beta
from math import sqrt, atan

def calculate_angle(alpha, beta):
  """
  Calculates the angle using the formula arcttan((sqrt(alpha))/(sqrt(beta)))

  Args:
      alpha_beta: A tuple containing alpha and beta values.

  Returns:
      The calculated angle in radians.
  """
  # Handle cases where alpha or beta is 0 to avoid division by zero
  if alpha == 0:
    if beta > 0:
      return pi/2  # 90 degrees
    elif beta < 0:
      return -pi/2  # -90 degrees
    else:
      return 0  # 0 degrees (indeterminate)
  elif beta == 0:
    return float('inf')  # Represents positive or negative infinity

  # Calculate the angle using the formula
  angle = atan(sqrt(alpha) / sqrt(beta))
  return angle


## Quantum Functions
This section details quantum operations essential for the algorithm:

- Quantum functions corresponding to the classical ones, allowing operations on quantum states based on classical pre-processing.

In [4]:
#TODO implement unitary_control() using Classiq's built in control()
@qfunc
def unitary_control(qubit: QArray[QBit], contrl: QArray[QBit], target: QParam[int]):
      control(lambda:X(qubit[target]),contrl )
      
#TODO implement y_rotation using the RY() gate
@qfunc
def y_rotation(theta: QParam[float], reg: QArray[QBit], target: QParam[int]):
      RY(theta, reg[target])

@qfunc
def my_controlled_unitary(q:QArray[QBit], w:QParam[float], ctrl:QArray[QBit], target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: unitary_control(q, ctrl, target))

@qfunc
def my_unitary(q:QArray[QBit], w:QParam[float],target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: X(q[target]))

## Sparse State Prep
Describes the arguments and setup for preparing a sparse quantum state:

- Initial setup includes defining quantum bit requirements and preparing the environment.
- Integration of classical functions to set up and manipulate quantum state preparation based on given sparse states.

### Algorithm 1
- **Purpose**: The primary function of `algorithm_1` is to efficiently prepare a sparse quantum state based on a given set of sparse state descriptions. It utilizes classical pre-processing to determine the optimal sequence of quantum gates.
- **Process**:
  - It begins by identifying the qubits and the operations that will result in the greatest simplification of the quantum state based on the input data.
  - The algorithm iteratively processes subsets of quantum states, toggling qubits and adjusting their probabilities to converge towards the desired sparse state.
  - Each iteration updates operation lists which are used to guide the quantum operations on the actual quantum hardware.
  

In [5]:

def algorithm_1(s,n, ops1, ops2, ops3, ops4, ops5, n9):
  dif_qubits = [] #Where to operate
  dif_values = [] #What operation

  #Preprocessing
  T = s

  dif_qubits, dif_values, t = process_subsets(T, n, dif_qubits, dif_values)
  
  dif = dif_qubits.pop()
  dif_values.pop()

  x_1 = t[0]
  t_dagger = [x for x in s if all(x[0][q] == v for q, v in zip(dif_qubits, dif_values))]
  t_dagger.remove(x_1)

  dif_qubits, dif_values, t_dagger = process_subsets(t_dagger, n, dif_qubits, dif_values)
  x_2 = t_dagger[0]


  #Storing necessary operations
  toggle_operations(dif,n,x_1,ops1,ops2,s)
  
  for b in range(n):
    if b != dif and x_1[0][b] != x_2[0][b]:
      conditional_toggle(ops1,ops3,n,dif,b,s)

  for b in dif_qubits:
    toggle_operations(b,n,x_2,ops1,ops2,s)

  alpha, beta = calc_alpha_beta(x_1,x_2)
  
  ops1 += [3]
  sy = [alpha,beta, dif_qubits, dif]

  if len(dif_qubits) > 0:
    n9 += [len(dif_qubits)]
  else:
    sy.remove(dif_qubits)

  ops4 += [sy]
  s.remove(x_1)

  if len(s) > 1:
    algorithm_1(s,n, ops1, ops2, ops3, ops4, ops5, n9)
  else:
    ops1 += [4]
    ops5 += [x_2[0]]

### Main Function Implementation
- **Purpose**: The `main` function orchestrates the overall setup and execution of the sparse state preparation.
- **Functionality**:
  - It initializes the quantum environment with the necessary number of qubits based on the input state descriptions.
  - The sparse states are first converted into a format suitable for quantum operations using `dict_to_3d_array`.
  - The main function then invokes `algorithm_1` to find the necessary operation to prepare the sparse quantum state.
  - After `algorithm_1` completes, the `main` function applies the quantum operations to finalize the state preparation.
  - It is responsible for managing and applying the computed operations to the quantum system, effectively translating the classical pre-processing results into quantum manipulations.

In [45]:
def process_states(current_state, ops1, ops2, ops3, q):

  for i in range(len(ops1)):
      op = ops1[i]

      # Update state based on operation type and apply to qubits
      if op == 1:  # NOT Gate (details in ops2[i])
          qubit_to_flip = ops2[i]
          X(psi[qubit_index])  # Apply NOT gate to corresponding qubit
      elif op == 2:  # CNOT Gate (details in ops3[i])
          control_index, target_index = ops3[i]
          CX(psi[control_index], psi[target_index])  # Apply CNOT gate
      elif op == 3:  # Custom Gate (details in ops4[i])
          # Not applicable in this algorithm (handled within algorithm1)
          pass
      elif op == 4:  # Final State (no further operations)
          pass

  

# Helper function to flip a bit in a state
def flip_bit(state, qubit_index):
  """
  Flips the bit at the specified qubit index in a state.

  Args:
      state: The state as a list representing a bitstring.
      qubit_index: The index of the qubit to flip.

  Returns:
      The updated state with the flipped bit.
  """
  new_state = state.copy()
  new_state[qubit_index] = 1 - new_state[qubit_index]
  return new_state




@qfunc
def main(psi: Output[QArray[QBit]]):
  

  #Arguments for algorithm1()
  sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
  NUM_QUBITS = len(next(iter(sparse_states)))

  ops1 = []#Stores operations
  ops2 = []
  ops3 = []
  ops4 = []
  ops5 = []
  n9 = []
  
  # **Allocate qubits for output and potentially an auxiliary qubit**
  allocate(NUM_QUBITS + 1,psi)  # Allocate for data qubits and one auxiliary
  sparse_states = dict_to_3d_array(sparse_states)




  # Check for consistent state lengths
  state_length = len(next(iter(sparse_states)))

  if len(sparse_states[0]) > 1:
    algorithm_1(sparse_states, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)
    
    # **TODO YOUR CODE HERE - Apply quantum gates based on ops lists**
    current_state = np.zeros(2**NUM_QUBITS, dtype=np.float64)
    print(len(current_state))
    while len(current_state) > 1:
        # Find a circuit to reduce state complexity using Algorithm 1
        new_ops1, new_ops2, new_ops3, new_ops4, new_ops5, new_n9 = algorithm_1(current_state, NUM_QUBITS, ops1, ops2, ops3, ops4, ops5, n9)

        # Update operations lists
        ops1.extend(new_ops1)
        ops2.extend(new_ops2)
        ops3.extend(new_ops3)
        ops4.extend(new_ops4)
        ops5.extend(new_ops5)
        n9.extend(new_n9)
        for i in range(len(ops1)):
           process_states(psi, ops1[i], ops2[i], ops3[i], psi)
  

         # Apply operations using allocated qubits
        for i in range(len(ops1)):
          op = ops1[i]
          if op == 1:  # Not gate
            qubit_index = ops2[i]
            X(psi[qubit_index])  # Use allocated register q
          elif op == 2:  # CNOT gate
            control_index, target_index = ops3[i]
            CX(psi[control_index], psi[target_index])  # Use allocated register q
          elif op == 3:  # Custom gate
            # Check if ops4 has enough values at index i
            if i < len(ops4) and len(ops4[i]) == 3:
              alpha, beta, qubit = ops4[i]
              theta = calculate_angle(alpha, beta)
              my_unitary(q=psi, w=theta, target=qubit)  # Use allocated register q
            else:
              print(f"Invalid ops4 data at index {i}")
              continue

       

        






        #TODO YOUR CODE ENDS HERE
  else:
      for b in range(NUM_QUBITS):
            if sparse_states[0][0][b]==1:
                X(psi[NUM_QUBITS-1-b])
         

model = create_model(main)
qprog = synthesize(model)
show(qprog)


8


IndexError: invalid index to scalar variable.

## Good Luck!

HINT: If you are getting inconsistent results when running code cells repeatedly, run all cells at once or restart your notebook kernel.